In [58]:
# Import Required Packages
%matplotlib inline
import numpy as np
#import networkx as nx
import time
import random
import json
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from sympy import *
from tqdm.notebook import tqdm

#From Qiskit library
from qiskit import *
from qiskit.visualization import *
from qiskit import IBMQ
from qiskit import BasicAer
from qiskit import Aer, execute
from qiskit.tools.visualization import plot_histogram, plot_state_city
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToQubo
from qiskit.quantum_info import Statevector
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit.circuit import Parameter
from qiskit.optimization.converters import IsingToQuadraticProgram
from qiskit.aqua.components.optimizers import NELDER_MEAD, CRS, ADAM

In [59]:
qiskit.qiskit.__qiskit_version__

{'qiskit-terra': '0.16.1',
 'qiskit-aer': '0.7.1',
 'qiskit-ignis': '0.5.1',
 'qiskit-ibmq-provider': '0.11.1',
 'qiskit-aqua': '0.8.1',
 'qiskit': '0.23.1'}

In [60]:
#Function: Load JSON file
def loadjson( filename ):
    loaded_file = open(filename, "r")
    json_data = json.load(loaded_file)
    loaded_file.close()
    return json_data

#Function: Load Pickle file
def loadpkl( filename ):
    loaded_file = open(filename, "rb")
    pkl_data = pickle.load( loaded_file )
    loaded_file.close()
    return pkl_data

#Function: Get single term coefficients (to 4 decimal places) from Sympy Expression 
def get_coeff( variable, expression ):
    poly = Poly(expression.coeff(variable), domain = 'RR')
    return round(float(poly.coeffs()[len(poly.coeffs())-1]), 4)

#Function: Get coefficient of quadratic term (to 4 decimal places) from Sympy Expression
def quad_coeff( var_1, var_2, expression ):
    return round(float(expression.coeff(var_1).coeff(var_2)), 4)

In [61]:
pwd

'/home/jedwin/trafficQAOA'

In [62]:
import sys
sys.executable

'/home/jedwin/anaconda3/envs/QC/bin/python'

In [63]:
#Read and store files into variables
car_routes = loadjson( "3cars3routes.json" )
# H = loadjson( "Hamiltonian9qbits.json")
# [objective, Ham, objective_normed, Ham_normed] = loadpkl("expressions.pkl")
[total_cost, constraints] = loadpkl("cost_constraints.pkl")

In [64]:
Q = IndexedBase('Q')
car_routes_variables = {}
for route in car_routes:
    car_routes_variables[route] = Q[route[3],route[10]]
car_routes_variables

{'car1_route1': Q[1, 1],
 'car1_route2': Q[1, 2],
 'car1_route3': Q[1, 3],
 'car2_route1': Q[2, 1],
 'car2_route2': Q[2, 2],
 'car2_route3': Q[2, 3],
 'car3_route1': Q[3, 1],
 'car3_route2': Q[3, 2],
 'car3_route3': Q[3, 3]}

In [65]:
no_routes = 3
no_cars = 3
no_qubits = no_routes * no_cars

In [66]:
#Associate binary variable to each Q11, Q12, etc with one index for QuadraticProgram class
binary_vars = {}
for variable in car_routes_variables:
    binary_vars[car_routes_variables[variable]] = 'X' + str(no_routes*(int(variable[3])-1)+int(variable[10]))
Qvariables = tuple(binary_vars.keys())
Xvariables = tuple(binary_vars.values())

In [67]:
#Construct Quadratic Problem using known (quadratic) objective function
penalty = 132*1.2
qubo1 = QuadraticProgram()
linear = []
for variable in car_routes_variables.values():
    qubo1.binary_var( binary_vars[variable] )
    linear.append( get_coeff(variable, total_cost) )
quadratic = {}
for i in range(no_qubits):
    for j in range(i+1, no_qubits):
        pairing = ( Xvariables[i], Xvariables[j] )
        coeff = quad_coeff( Qvariables[i], Qvariables[j], total_cost )
        if int(coeff) != 0:
            quadratic[pairing] = coeff
qubo1.minimize(linear = linear, quadratic = quadratic)

#Add constraints
for car in constraints:
    i = 3*(int(car[3])-1) #to specify index of variable on next line
    qubo1.linear_constraint(linear={'X'+str(i+1): 1, 'X'+str(i+2): 1, 'X'+str(i+3): 1}, sense='==', rhs=1, name=str(car))

#Convert constrained Quadratic Problem into QUBO
converter = QuadraticProgramToQubo(penalty = penalty)
qubo1 = converter.convert(qubo1)

In [68]:
op, offset = qubo1.to_ising()

In [69]:
coeffs = [abs(op.oplist[i].coeff) for i in range(len(op.oplist))]
norm_factor = np.amax(coeffs)

In [70]:
#Normalised objective function
qubo = QuadraticProgram()
linear = []
for variable in car_routes_variables.values():
    qubo.binary_var( binary_vars[variable] )
    linear.append( (get_coeff(variable, total_cost)/norm_factor) )
quadratic = {}
for i in range(no_qubits):
    for j in range(i+1, no_qubits):
        pairing = ( Xvariables[i], Xvariables[j] )
        coeff = quad_coeff( Qvariables[i], Qvariables[j], total_cost )
        if int(coeff) != 0:
            quadratic[pairing] = (coeff/norm_factor)
qubo.minimize(linear = linear, quadratic = quadratic)

#Add constraints
for car in constraints:
    i = 3*(int(car[3])-1) #to specify index of variable on next line
    qubo.linear_constraint(linear={'X'+str(i+1): 1, 'X'+str(i+2): 1, 'X'+str(i+3): 1}, sense='==', rhs=1, name=str(car))

#Convert constrained Quadratic Problem into QUBO
converter = QuadraticProgramToQubo(penalty = penalty/norm_factor)
qubo = converter.convert(qubo)

In [81]:
op, offset = qubo.to_ising()

In [72]:
qaoa_mes = QAOA(quantum_instance=Aer.get_backend('statevector_simulator'))
exact_mes = NumPyMinimumEigensolver()
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [73]:
%%time
exact_result = exact.solve(qubo)
print(exact_result)

optimal function value: 1.2999467234949391
optimal value: [0. 1. 0. 0. 1. 0. 1. 0. 0.]
status: SUCCESS
CPU times: user 89.8 ms, sys: 4.09 ms, total: 93.9 ms
Wall time: 91.1 ms


In [82]:
#Binary string x of length 9, using cost/obj function from QUBO on QuadraticProgram qp.
def eval_cost(x, qubo_problem):
    return qubo_problem.objective.evaluate([int(x[no_qubits-1-i]) for i in range(no_qubits)])

In [23]:
#Print first 20 lowest states
values = {}
for k in range(2**9):
    string = '{0:09b}'.format(k)
    values[string] = eval_cost(string, qubo)
sort_values = sorted(values.items(), key=lambda x: x[1])
i=0
for string, cost in sort_values:
    print(string, round(cost,3), string[0:3].count('1')==1 and string[3:6].count('1')==1 and string[6:9].count('1')==1)
    i += 1
    if i == 20:
        break

001010010 1.3 True
001001010 1.364 True
100001010 1.502 True
001010001 1.513 True
010001010 1.534 True
100010010 1.545 True
010010001 1.556 True
010010010 1.566 True
000001010 1.568 False
001000010 1.568 False
001100010 1.577 True
001001100 1.588 True
010001100 1.63 True
100010001 1.63 True
001010100 1.63 True
100001100 1.705 True
000010001 1.718 False
001010000 1.718 False
010010100 1.769 True
000001100 1.792 False


In [91]:
#Use statevector with GPU to simulate slightly faster for less than ~25 qubits
#If more than 25 qubits, use method = "statevector" (CPU) instead.

simulator = QasmSimulator()
optimizer = NELDER_MEAD(maxiter = 2)

In [92]:
#Construct QAOA circuit of size p given operator from qubo problem ( operator is obtained from QuadraticProblem.to_ising() )
def construct_QAOAcirc( p, operator ):
    #Initialise parameter list
    params_expr = []
    
    #Create arbitrary "gamma_n" parameters
    for a in range(p):
        params_expr.append( Parameter('a'+str(a+1)) )
    #Likewise for "beta_n" parameters
    for b in range(p):
        params_expr.append( Parameter('b'+str(b+1)) )
        
    #Create circuit given operator, gamma, and beta parameters
    qaoa = QAOA(operator = operator, p=p, quantum_instance=Aer.get_backend('qasm_simulator'))
    circuit = qaoa.construct_circuit( params_expr[0:2*p] )[0]
    circuit.measure_active()
    return p, params_expr, circuit


#Find expectation value, given qaoa_circuit, qubo_problem and freq_params (u1, u2, ... , v1, v2, ...)
def expectation( qaoa_circuit, qubo_problem, freq_params ):
    
    #unload p, params_expr, circuit from the return value of contruct_QAOAcirc
    p = int(qaoa_circuit[0])
    params_expr = qaoa_circuit[1]
    circuit = qaoa_circuit[2]
    params = convert_fourier_parameters(p,p,freq_params)
    if len(params) != 2*p:
        print("Must have len(params) = 2*no_layers")
    else:    
        circuit2 = circuit.assign_parameters({params_expr[i]: params[i] for i in range(len(params))}, inplace = False)
        no_shots = 100000
        result = execute(circuit2, simulator, shots = no_shots).result()
        counts = result.get_counts()
        exp_val = 0
        for state in counts:
            a = counts[state]
            b = eval_cost(state, qubo_problem)
            exp_val += a*b        
        exp_val /= no_shots
        print(".", end = "")
        return np.round(exp_val, decimals = 1)

#Convert frequency parameters to standard gamma and beta parameters
def convert_fourier_parameters(p , q, frequency_parameters):
    u_params = np.array(frequency_parameters[0:q], dtype = float)
    v_params = np.array(frequency_parameters[q:2*q], dtype = float)
    gamma_params = np.empty((p,), dtype = np.float32)
    beta_params = np.empty((p,), dtype = np.float32)
    for i in range(p):
        gamma_inter, beta_inter = 0,0
        a = (i+0.5)*np.pi/p
        for k in range(q):
            gamma_inter += u_params[k]*np.sin( (k+0.5)*a )
            beta_inter += v_params[k]*np.cos( (k+0.5)*a )
        gamma_params[i] = gamma_inter
        beta_params[i] = beta_inter
    return np.append(gamma_params, beta_params)
    
#Binary string x of length 9, using cost/obj function from QUBO on QuadraticProgram qp.
def eval_cost(x, qubo_problem):
    y = [int(x[no_qubits-1-i]) for i in range(no_qubits)] #reverse order
    value = qubo_problem.objective.evaluate(y)
    return value

Inputs: QAOA problem, no_layers, u_v_L, u_v_B, no_perturb

1. Unload operator from QAOA problem
2. Construct circuit with parameters for no_layers
3. Generate no_perturb number of parameters about u_v_B
4. Optimize around the u_v_L, u_v_B, and the perturbed u_v_B's
5. Choose best for the new u_v_B, and also the new u_v_L
6. Return the new u_v_B, and the new u_v_B

Note: Make sure u_v_L = (u1, u2, ..., v1, v2, ...) has 2*(no_layers - 1) 

In [93]:
def optimize_qubo(qubo_problem, p, freq_optimal_params_L, freq_optimal_params_B, no_perturb):
    perturb_scale_factor = 0.6
    op, offset = qubo_problem.to_ising()
    generator = np.random.default_rng()
    variable_bounds = [ (-np.pi/2, np.pi/2) for _ in range(2*p)]
    if len(freq_optimal_params_L) != len(freq_optimal_params_B) and len(freq_optimal_params_B) != (p-1)*2:
        print('Check number of input parameters are correct')
        return None
    
    #Construct circuit
    print(f'Step {p}: Building circuit...')
    circuit = construct_QAOAcirc(p, op)
    
    #New initial points generated from previous initial points
    params = np.zeros(shape = (2*p,), dtype=float)+0.05  
    initial_freq_params_L = params.copy()
    initial_freq_params_B_0 = params.copy()
    initial_freq_params_L[0:p-1] = freq_optimal_params_L[0:p-1]
    initial_freq_params_L[p:2*p-1] = freq_optimal_params_L[p-1:2*p-2]
    initial_freq_params_B_0[0:p-1] = freq_optimal_params_B[0:p-1]
    initial_freq_params_B_0[p:2*p-1] = freq_optimal_params_B[p-1:2*p-2]
    
    #Generate perturbed initial points     
    if p != 1:
        initial_freq_params_B = np.empty(shape = (no_perturb+2, ), dtype = object)
        initial_freq_params_B[0] = initial_freq_params_B_0
        u_random = np.array([np.array(generator.normal(0, freq_optimal_params_B[i]**2, no_perturb+1)) for i in range(0,p-1)])
        v_random = np.array([np.array(generator.normal(0, freq_optimal_params_B[i]**2, no_perturb+1)) for i in range(p-1,2*p-2)])
        m = len(initial_freq_params_B)
        for j in range(1,m-1):
            perturbed_params = params.copy()
            perturbed_params[0:p-1] = initial_freq_params_B_0[0:p-1] + perturb_scale_factor*u_random[:,j]
            perturbed_params[p:2*p-1] = initial_freq_params_B_0[p:2*p-1] + perturb_scale_factor*v_random[:,j]
            initial_freq_params_B[j] = perturbed_params
        initial_freq_params_B[m-1] = initial_freq_params_B[0]
        
    print(f'Step {p}: Now optimising parameters (Nelder-Mead)...')
    ##Define obj_function that inputs only parameters
    obj_expectation = lambda x: expectation( circuit, qubo_problem, x ) 
    print('\n'+'initial freq point L:', initial_freq_params_L)
    optimal_result = optimizer.optimize(num_vars = 2*p,
                                        objective_function = obj_expectation,
                                        variable_bounds = variable_bounds, 
                                        initial_point = initial_freq_params_L)
    optimal_params_L, exp_val_L, no_evals = optimal_result
    
    if p == 1:
        optimal_params_B, exp_val_B = optimal_params_L, exp_val_L
    
    else:
        perturbed_params = [optimal_params_L]
        perturbed_exp_vals = [exp_val_L]
        print(initial_freq_params_B)
        for l in range(m):
            print('\n'+f'{l}, initial freq point B: ', initial_freq_params_B[l])
            if l == m-1:
                break
            perturbed_optimal_result = optimizer.optimize(num_vars = 2*p,
                                    objective_function = obj_expectation,
                                    variable_bounds= variable_bounds, 
                                    initial_point=initial_freq_params_B[l])
            print('\nNelderMeadcomplete')
            perturbed_optimal_params, perturbed_optimal_expval = perturbed_optimal_result[0:2] 
            perturbed_params.append(perturbed_optimal_params)
            perturbed_exp_vals.append(perturbed_exp_vals)
        exp_val_B = np.amin(perturbed_exp_vals)
        minim_index = np.awhere(perturbed_exp_vals == exp_val_B)[0][0]
        optimal_params_B = perturbed_params[minim_index]
    
    return optimal_params_L, optimal_params_B

In [94]:
optimal_params_L_1, optimal_params_B_1 = optimize_qubo(qubo, 1, [], [], 1)

Step 1: Building circuit...
Step 1: Now optimising parameters (Nelder-Mead)...

initial freq point L: [0.05 0.05]
.......

In [ ]:
optimal_params_L_2, optimal_params_B_2 = optimize_qubo(qubo, 2, optimal_params_L_1, optimal_params_B_1, 1)

Step 2: Building circuit...
Step 2: Now optimising parameters (Nelder-Mead)...

initial freq point L: [0.05 0.05 0.05 0.05]
......[array([0.05, 0.05, 0.05, 0.05])
 array([0.05003811, 0.05      , 0.04901847, 0.05      ])
 array([0.05, 0.05, 0.05, 0.05])]

0, initial freq point B:  [0.05 0.05 0.05 0.05]
.......
NelderMeadcomplete

1, initial freq point B:  [0.05003811 0.05       0.04901847 0.05      ]
.....

In [ ]:
print(optimal_params_L_1)
print(optimal_params_B_1)

In [29]:
a = construct_QAOAcirc(1, op)

In [30]:
a[0]

1

In [32]:
simulator = QasmSimulator(method = "statevector_gpu")

In [34]:
%%time
expectation(a, qubo, np.array([0.05,0.05]))

.CPU times: user 556 ms, sys: 0 ns, total: 556 ms
Wall time: 556 ms


5.6

In [37]:
op

SummedOp([PauliOp(Pauli(z=[True, False, False, False, False, False, False, False, False], x=[False, False, False, False, False, False, False, False, False]), coeff=-0.9041022908897177), PauliOp(Pauli(z=[False, True, False, False, False, False, False, False, False], x=[False, False, False, False, False, False, False, False, False]), coeff=-0.6670218433670753), PauliOp(Pauli(z=[False, False, True, False, False, False, False, False, False], x=[False, False, False, False, False, False, False, False, False]), coeff=-0.7682472029834843), PauliOp(Pauli(z=[False, False, False, True, False, False, False, False, False], x=[False, False, False, False, False, False, False, False, False]), coeff=-0.9041022908897177), PauliOp(Pauli(z=[False, False, False, False, True, False, False, False, False], x=[False, False, False, False, False, False, False, False, False]), coeff=-0.7362812999467236), PauliOp(Pauli(z=[False, False, False, False, False, True, False, False, False], x=[False, False, False, False,

In [43]:
def eval_cost(x, linear, quadratic):
    x1 = int(x[0])
    x2 = int(x[1])
    x3 = int(x[2])
    x4 = int(x[3])
    x5 = int(x[4])
    x6 = int(x[5])
    x7 = int(x[6])
    x8 = int(x[7])
    x9 = int(x[8])
    cost = 0
    

X1
X2
X3
X4
X5
X6
X7
X8
X9


In [77]:
qubo

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 1.368140649973 X1 - 1.282898241875 X2 - 1.059136920618 X3
      - 1.368140649973 X4 - 1.133724027704 X5 - 1.048481619606 X6
      - 1.368140649973 X7 - 1.229621736814 X8 - 1.144379328716 X9 + [
      1.687799680341 X1^2 + 3.375599360682 X1*X2 + 3.375599360682 X1*X3
      + 0.639318060735 X1*X4 + 0.426212040490 X1*X6 + 0.639318060735 X1*X7
      + 0.447522642515 X1*X8 + 0.426212040490 X1*X9 + 1.687799680341 X2^2
      + 3.375599360682 X2*X3 + 0.042621204049 X2*X5 + 0.255727224294 X2*X8
      + 0.042621204049 X2*X9 + 1.687799680341 X3^2 + 0.255727224294 X3*X5
      + 1.687799680341 X4^2 + 3.375599360682 X4*X5 + 3.801811401172 X4*X6
      + 0.639318060735 X4*X7 + 0.447522642515 X4*X8 + 0.426212040490 X4*X9
      + 1.687799680341 X5^2 + 3.375599360682 X5*X6 + 1.687799680341 X6^2
      + 0.426212040490 X6*X7 + 0.362280234417 X6*X8 + 0.426212040490 X6*X9
      + 1.687799680341 X7^2 + 3.8231

In [80]:
qubo

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 1.368140649973 X1 - 1.282898241875 X2 - 1.059136920618 X3
      - 1.368140649973 X4 - 1.133724027704 X5 - 1.048481619606 X6
      - 1.368140649973 X7 - 1.229621736814 X8 - 1.144379328716 X9 + [
      1.687799680341 X1^2 + 3.375599360682 X1*X2 + 3.375599360682 X1*X3
      + 0.639318060735 X1*X4 + 0.426212040490 X1*X6 + 0.639318060735 X1*X7
      + 0.447522642515 X1*X8 + 0.426212040490 X1*X9 + 1.687799680341 X2^2
      + 3.375599360682 X2*X3 + 0.042621204049 X2*X5 + 0.255727224294 X2*X8
      + 0.042621204049 X2*X9 + 1.687799680341 X3^2 + 0.255727224294 X3*X5
      + 1.687799680341 X4^2 + 3.375599360682 X4*X5 + 3.801811401172 X4*X6
      + 0.639318060735 X4*X7 + 0.447522642515 X4*X8 + 0.426212040490 X4*X9
      + 1.687799680341 X5^2 + 3.375599360682 X5*X6 + 1.687799680341 X6^2
      + 0.426212040490 X6*X7 + 0.362280234417 X6*X8 + 0.426212040490 X6*X9
      + 1.687799680341 X7^2 + 3.8231

In [86]:
x = '110100010'

In [87]:
[int(x[no_qubits-1-i]) for i in range(no_qubits)]

[0, 1, 0, 0, 0, 1, 0, 1, 1]